In [1]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CherryLeavesDataset(Dataset):
    def __init__(self, root_dir, split, category, transform=None):
        self.root_dir = os.path.join(root_dir, split, category)
        self.transform = transform
        self.images = [f for f in os.listdir(self.root_dir) if f.endswith(('.jpg', '.JPG', '.jpeg', '.JPEG'))]
        self.labels = 0 if category == "healthy" else 1  # 0 for healthy, 1 for powdery_mildew

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor, already normalized [0,1] in 02
])

base_dir = "../inputs/cherry-leaves_dataset/"

train_healthy = CherryLeavesDataset(base_dir, "train", "healthy", transform)
train_mildew = CherryLeavesDataset(base_dir, "train", "powdery_mildew", transform)
val_healthy = CherryLeavesDataset(base_dir, "validation", "healthy", transform)
val_mildew = CherryLeavesDataset(base_dir, "validation", "powdery_mildew", transform)
test_healthy = CherryLeavesDataset(base_dir, "test", "healthy", transform)
test_mildew = CherryLeavesDataset(base_dir, "test", "powdery_mildew", transform)

train_dataset = torch.utils.data.ConcatDataset([train_healthy, train_mildew])
val_dataset = torch.utils.data.ConcatDataset([val_healthy, val_mildew])
test_dataset = torch.utils.data.ConcatDataset([test_healthy, test_mildew])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 1472
Validation dataset size: 316
Test dataset size: 316


In [2]:
class MildewCNN(nn.Module):
    def __init__(self):
        super(MildewCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Input: 3 channels (RGB), Output: 16 filters
        self.pool = nn.MaxPool2d(2, 2)  # Reduce spatial dimensions by 2
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # Output: 32 filters
        self.fc1 = nn.Linear(32 * 64 * 64, 120)  # Flatten to 32 * 64 * 64 = 131,072
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)  # 2 classes: healthy, powdery_mildew

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten dynamically, should be [batch_size, 131,072]
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = MildewCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Model, loss, and optimizer initialized!")




Model, loss, and optimizer initialized!


In [3]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

train_model(model, train_loader, criterion, optimizer)

Epoch [1/1], Loss: 0.5592, Accuracy: 70.92%


In [4]:
def validate_model(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(val_loader)
    val_acc = 100 * correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

validate_model(model, val_loader, criterion)

Validation Loss: 0.1957, Validation Accuracy: 94.30%


In [5]:
train_model(model, train_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Loss: 0.0982, Accuracy: 96.88%
Epoch [2/5], Loss: 0.0592, Accuracy: 97.83%
Epoch [3/5], Loss: 0.0141, Accuracy: 99.59%
Epoch [4/5], Loss: 0.0080, Accuracy: 99.86%
Epoch [5/5], Loss: 0.0042, Accuracy: 100.00%


In [6]:
def test_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_loss = running_loss / len(test_loader)
    test_acc = 100 * correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')

test_model(model, test_loader, criterion)

Test Loss: 0.0237, Test Accuracy: 99.37%
